<a href="https://colab.research.google.com/github/sergope/tfm/blob/main/error_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cálculo RMSE

from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd
import numpy as np

In [ ]:
def filter_by_date():
  dates = pd.date_range(start = date_init, end = date_end, freq='D').strftime('%Y-%m-%d')
  merge_dates = merged_df[merged_df['ds'].isin(list(dates))]
  forecast_dates = forecast_df[forecast_df['ds'].isin(list(dates))]


In [ ]:
# Cálculo RMSE
def calculate_rmse(merged_df, forecast_df, date_init, date_end):
  dates = pd.date_range(start = date_init, end = date_end, freq='D').strftime('%Y-%m-%d')
  merge_dates = merged_df[merged_df['ds'].isin(list(dates))]
  forecast_dates = forecast_df[forecast_df['ds'].isin(list(dates))]

  y_actual = merge_dates['y']
  y_predicted = forecast_dates['yhat']

  rmse = sqrt(mean_squared_error(y_actual, y_predicted))
  return rmse

In [ ]:
# Check True and False positives

def date_list(df_merged, df_forecast, init, end):
  date_ini = df_merged['ds'][init].strftime('%Y-%m-%d')
  date_end = df_merged['ds'][init+6].strftime('%Y-%m-%d')

  if date_end == df_merged['ds'][end].strftime('%Y-%m-%d'):
    return []

  list_dates = pd.date_range(start =date_ini, end = date_end, freq='D').strftime('%Y-%m-%d')
  return list_dates

def add_mean_to_row(row):
  return (row['yhat'] + row['yhat_upper']) / 2

def check_warnings(df1, df2, limit):
  evaluation = {
      'true_positive': False,
      'false_positive': False,
      'true_negative': False,
      'false_negative': False,
      
  }

  len_df1 = len(df1[df1['y'] >= limit ])
  df2['yhat_mean'] = df2.apply(lambda row: add_mean_to_row(row), axis=1)
  len_df2 = len(df2[df2['yhat_mean'] >= limit ])

  if len_df1 == len_df2:
    if len_df1 == 0:
      evaluation['true_negative'] = True
      return evaluation
    else:
      evaluation['true_positive'] = True
      return evaluation

  else:
    if len_df1 > len_df2:
      evaluation['false_negative'] = True
      return evaluation
    else:
      evaluation['false_positive'] = True
      return evaluation


def check_week(merged_df, forecast_df, date_init, date_end, limit):
  true_positive = 0
  false_positive = 0
  true_negative = 0
  false_negative = 0

  dates = pd.date_range(start = date_init, end = date_end, freq='D').strftime('%Y-%m-%d')
  merge_dates = merged_df[merged_df['ds'].isin(list(dates))]
  forecast_dates = forecast_df[forecast_df['ds'].isin(list(dates))]

  forecast_dates.reset_index(drop=True, inplace=True)
  merge_dates.reset_index(drop=True, inplace=True)

  for i in np.arange(0,merge_dates.count()[0]):
    init = i
    end = merge_dates.count()[0] - 1 
    dates = date_list(merge_dates, forecast_dates, init, end)

    if not len(dates):
      print('true_positive = ' + str(true_positive))
      print('true_negative = ' + str(true_negative))
      print('false_positive = ' + str(false_positive))
      print('false_negative = ' + str(false_negative))
      return

    merged_copy = merge_dates[merge_dates['ds'].isin(list(dates))].copy()
    forecast_copy = forecast_dates[forecast_dates['ds'].isin(list(dates))].copy()

    result = check_warnings_2(merged_copy, forecast_copy, limit)

    if result['true_positive']:
      true_positive += 1
      pass
    elif result['false_positive']:
      false_positive += 1
      pass
    elif result['true_negative']:
      true_negative += 1
      pass
    elif result['false_negative']:
      false_negative += 1
      pass